In [ ]:
import os
import subprocess
from pathlib import Path

END_WITH_LOCAL = 'archive'

os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

BASE_DIR = os.getcwd()
print(f"BASE_DIR: {BASE_DIR}")

# Simple validation
if not (BASE_DIR.endswith('/content') or BASE_DIR.endswith(END_WITH_LOCAL)):
    raise ValueError(f"Expected to be in .../{END_WITH_LOCAL} or .../content directory, but got: {BASE_DIR}")

BASE_DIR: /Users/tungnguyen/Library/CloudStorage/GoogleDrive-nguyenlamtungthptltt@gmail.com/My Drive/Projects-Large/Group-Project-B3/code-just-to-preview/characters-and-dialouges-association-in-comics/code/bubble-detection


In [13]:
import os
from pathlib import Path

Manga109_dir = os.path.join(BASE_DIR,'../../data/Manga109/Manga109/Manga109_released_2023_12_07/Manga109_released_2023_12_07/images')


In [14]:
folders = [f.name for f in Path(Manga109_dir).iterdir() if f.is_dir()]
print(folders)

['NichijouSoup', 'GakuenNoise', 'TsubasaNoKioku', 'JangiriPonpon', 'That_sIzumiko', 'BokuHaSitatakaKun', 'GinNoChimera', 'GarakutayaManta', 'MagicStarGakuin', 'UchiNoNyan_sDiary', 'ToutaMairimasu', 'Akuhamu', 'EvaLady', 'UltraEleven', 'Nekodama', 'OhWareraRettouSeitokai', 'HighschoolKimengumi_vol20', 'YamatoNoHane', 'YouchienBoueigumi', 'AppareKappore', 'HeiseiJimen', 'MukoukizuNoChonbo', 'HanzaiKousyouninMinegishiEitarou', 'SaladDays_vol01', 'UnbalanceTokyo', 'TotteokiNoABC', 'HaruichibanNoFukukoro', 'GOOD_KISS_Ver2', 'PlatinumJungle', 'SyabondamaKieta', 'PrayerHaNemurenai', 'MoeruOnisan_vol19', 'SamayoeruSyonenNiJunaiWo', 'TapkunNoTanteisitsu', 'JijiBabaFight', 'MayaNoAkaiKutsu', 'AosugiruHaru', 'ByebyeC-BOY', 'PrismHeart', 'Donburakokko', 'HinagikuKenzan', 'EienNoWith', 'HisokaReturns', 'BEMADER_P', 'MeteoSanStrikeDesu', 'Saisoku', 'TasogareTsushin', 'WarewareHaOniDearu', 'ShimatteIkouze_vol01', 'YumeiroCooking', 'UchuKigekiM774', 'TetsuSan', 'ParaisoRoad', 'Joouari', 'YumeNoKayoiji

In [15]:
folders = sorted([f.name for f in Path(Manga109_dir).iterdir() if f.is_dir()])
print(folders)

['ARMS', 'AisazuNihaIrarenai', 'AkkeraKanjinchou', 'Akuhamu', 'AosugiruHaru', 'AppareKappore', 'Arisa', 'BEMADER_P', 'BakuretsuKungFuGirl', 'Belmondo', 'BokuHaSitatakaKun', 'BurariTessenTorimonocho', 'ByebyeC-BOY', 'Count3DeKimeteAgeru', 'DollGun', 'Donburakokko', 'DualJustice', 'EienNoWith', 'EvaLady', 'EverydayOsakanaChan', 'GOOD_KISS_Ver2', 'GakuenNoise', 'GarakutayaManta', 'GinNoChimera', 'Hamlet', 'HanzaiKousyouninMinegishiEitarou', 'HaruichibanNoFukukoro', 'HarukaRefrain', 'HealingPlanet', 'HeiseiJimen', 'HighschoolKimengumi_vol01', 'HighschoolKimengumi_vol20', 'HinagikuKenzan', 'HisokaReturns', 'JangiriPonpon', 'JijiBabaFight', 'Joouari', 'Jyovolley', 'KarappoHighschool', 'KimiHaBokuNoTaiyouDa', 'KoukouNoHitotachi', 'KuroidoGanka', 'KyokugenCyclone', 'LancelotFullThrottle', 'LoveHina_vol01', 'LoveHina_vol14', 'MAD_STONE', 'MadouTaiga', 'MagicStarGakuin', 'MagicianLoad', 'MariaSamaNihaNaisyo', 'MayaNoAkaiKutsu', 'MemorySeijin', 'MeteoSanStrikeDesu', 'MiraiSan', 'MisutenaideDaisy'

In [16]:
original_image_path = []

# Get the first 30 sorted (case-sensitive) folders (volumes)
first_30_volumes = sorted([f for f in Path(Manga109_dir).iterdir() if f.is_dir()], key=lambda x: x.name)[:30]

# For each volume, get the first 11 images sorted in ascending order
for volume in first_30_volumes:
    images = sorted([f for f in volume.iterdir() if f.is_file() and f.suffix.lower() == '.jpg'], key=lambda x: x.name)
    for img_path in images[:16]:
        original_image_path.append(str(img_path))

print(len(original_image_path))

human_annotate_dir = os.path.join(BASE_DIR,'../../data/Human_Annotate_300/train')

all_img_paths = []

# for root, dirs, files in os.walk(human_annotate_dir):
#     for file in files:
#         if file.lower().endswith(('.jpg', '.jpeg', '.png')):
#             all_img_paths.append(os.path.join(root, file))

# Only scan immediate directory (no subdirectories)
all_img_paths = []
for file in os.listdir(human_annotate_dir):
    file_path = os.path.join(human_annotate_dir, file)
    if os.path.isfile(file_path) and file.lower().endswith(('.jpg', '.jpeg', '.png')):
        all_img_paths.append(file_path)

print(len(all_img_paths))

for volume in first_30_volumes:
    os.makedirs(os.path.join(human_annotate_dir, volume.name), exist_ok=True)

448
19


In [17]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time

# def compare_images(path1, path2):
#     img1 = cv2.imread(path1)
#     img2 = cv2.imread(path2)

#     if img1 is None or img2 is None:
#         return False
#     if img1.shape != img2.shape:
#         return False
#     if np.array_equal(img1, img2):
#         plt.figure(figsize=(10, 5))
#         plt.subplot(1, 2, 1)
#         plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
#         plt.title('Image Original to compare')
#         plt.subplot(1, 2, 2)
#         plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
#         plt.title('Image in Human Annotate')
#         plt.show()
#         time.sleep(0.2)
#         plt.close()
#         return True
#     return False

def compare_images(path1, path2, threshold=0.95):
    img1 = cv2.imread(path1)
    img2 = cv2.imread(path2)

    if img1 is None or img2 is None:
        return False
    
    # If shapes are different, try to resize to match
    if img1.shape != img2.shape:
        # Resize img2 to match img1's dimensions
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # Method 1: Structural Similarity Index (SSIM)
    from skimage.metrics import structural_similarity as ssim
    
    # Convert to grayscale for SSIM
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    similarity_score = ssim(gray1, gray2)
    
    if similarity_score >= threshold:
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
        plt.title('Image Original to compare')
        plt.subplot(1, 3, 2)
        plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
        plt.title('Image in Human Annotate')
        plt.subplot(1, 3, 3)
        plt.text(0.5, 0.5, f'SSIM Score: {similarity_score:.4f}', 
                 horizontalalignment='center', verticalalignment='center', 
                 transform=plt.gca().transAxes, fontsize=14)
        plt.axis('off')
        plt.show()
        time.sleep(0.2)
        plt.close()
        return True
    
    return False
        
    

In [18]:
import shutil
import os
from tqdm import tqdm

count_found = 0
original_image_path_len = len(original_image_path)
print(f"Total original images to find: {original_image_path_len}")
all_img_paths_len = len(all_img_paths)
print(f"Total images in human annotate directory: {all_img_paths_len}")

for img_path in tqdm(original_image_path, desc="Processing original images"):
    copy_img_path = os.path.join(human_annotate_dir, Path(img_path).parent.name)
    copy_img_name = Path(img_path).name
    for img_human in all_img_paths:
        if compare_images(img_path, img_human):
            found = True
            # Copy and rename the image
            shutil.copy(img_human, os.path.join(copy_img_path, copy_img_name))
            # Copy and rename the XML if it exists
            xml_human = os.path.splitext(img_human)[0] + ".xml"
            if os.path.exists(xml_human):
                shutil.copy(xml_human, os.path.join(copy_img_path, os.path.splitext(copy_img_name)[0] + ".xml"))
                os.remove(xml_human)  # Remove the xml file after copying
            # Remove img_human from all_img_paths and delete the file
            all_img_paths.remove(img_human)
            os.remove(img_human)  # Remove the image file after copying
            count_found += 1
            break
        else:
            continue

Total original images to find: 448
Total images in human annotate directory: 19


Processing original images: 100%|██████████| 448/448 [09:37<00:00,  1.29s/it]


In [19]:
print(count_found)

0
